In [1]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import re
import json

## Data Load

In [ ]:
text = "text"

In [2]:
all_data = load_dataset('csv',data_files='./order_speech_ko.csv',split='train')

Using custom data configuration default-8e0d0b8910acc620


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to C:\Users\Mu-jun\.cache\huggingface\datasets\csv\default-8e0d0b8910acc620\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


## Data Cleaning

In [3]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch[text] = re.sub(chars_to_ignore_regex, '', batch[text]).lower() + " "
    return batch

In [4]:
remove_spectial_char_data = all_data.map(remove_special_characters)

Parameter 'function'=<function remove_special_characters at 0x0000026AE756F8B8> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/142367 [00:00<?, ?ex/s]

## Split to Font

In [5]:
def extract_all_chars(batch):
    all_text = " ".join(batch[text])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


In [6]:
char_vocab = remove_spectial_char_data.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=remove_spectial_char_data.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

## Add index

In [7]:
vocab_list = list(set(char_vocab["vocab"][0]))

In [8]:
vocab_dict = {v: k+5 for k, v in enumerate(vocab_list)}

## Add Special Token

In [9]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["<pad>"] = 0
vocab_dict["<s>"] = 1
vocab_dict["</s>"] = 2
vocab_dict["<unk>"] = 3
len(vocab_dict)

602

## Export Vocabulary to json file

In [10]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)